# Word Sense Disambiguation (снятие лексической неоднозначности) и Word Sense Induction (нахождение значений слова)

In [1]:
import adagram
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))



def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize(text):
    
    words = tokenize(text)
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

## Задание № 1. Протестировать адаграм в определении перефразирования

In [2]:
vm = adagram.VectorModel.load("out.pkl")

In [10]:
v = vm.sense_vector("мир", 0)

In [89]:
v.shape

(100,)

In [13]:
import pandas as pd
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [14]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [15]:
data[:5]

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,"[полицейский, разрешить, стрелять, поражение, ...","[полиция, мочь, разрешить, стрелять, хулиган, ..."
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,"[право, полицейский, проникновение, жилища, ре...","[правило, внесудебный, проникновение, полицейс..."
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,"[президент, египет, ввести, чрезвычайный, поло...","[власть, египет, угрожать, ввести, страна, чре..."
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,"[вернуться, сирия, россиянин, волновать, вопро...","[самолёт, мчс, вывезти, россиянин, разрушить, ..."
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,"[москва, сирия, вернуться, 2, самолёт, мчс, ро...","[самолёт, мчс, вывезти, россиянин, разрушить, ..."


Векторизуйте пары текстов с помощью Адаграма, обучите любую модель и оцените качество (кросс-валидацией). 

За основу возьмите код из предыдущего семинара/домашки, только в функции 
get_embedding вам нужно выбирать вектор нужного значения (импользуйте model.disambiguate и model.sense_vector). Отдельные векторы усредните как и в предыдущем семинаре.

Для вытаскивания пар (целевое слово, контекстые слова) вам нужно будет написать специальную функцию.

In [62]:
# words = [0,1,2,3,4,5,6,7,8,9]
def get_words_in_context(words, window=3):
    words_context = []
    for i, w in enumerate(words):
        context = [words[c] for c in range(i - window, i + window + 1) if c != i and  c >= 0 and c < len(words)]  
        words_context.append([w, context])
    return words_context

In [61]:
# работать должно вот так
get_words_in_context(words)

[[0, [1, 2, 3]],
 [1, [0, 2, 3, 4]],
 [2, [0, 1, 3, 4, 5]],
 [3, [0, 1, 2, 4, 5, 6]],
 [4, [1, 2, 3, 5, 6, 7]],
 [5, [2, 3, 4, 6, 7, 8]],
 [6, [3, 4, 5, 7, 8, 9]],
 [7, [4, 5, 6, 8, 9]],
 [8, [5, 6, 7, 9]],
 [9, [6, 7, 8]]]

Когда получиться такой же результат, добавьте эту функцию в get_embedding. Проходите циклом по результату работы get_words_in_context и поставляйте каждый элемент-список в model.disambiguate.

In [63]:
def get_embedding_adagram(text, model, window, dim):
    
    word2context = get_words_in_context(text, window)
    
    vectors = np.zeros((len(word2context), dim))
    
    for i, (word, context) in enumerate(word2context):
        
        try:
            index = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, index)
            vectors[i] = v
        
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
        

In [83]:
dim = 100
X_text_1_vm = np.zeros((len(data['text_1_norm']), dim))
X_text_2_vm = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_vm[i] = get_embedding_adagram(text, vm, 3, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_vm[i] = get_embedding_adagram(text, vm, 3, dim)

In [84]:
X_text_vm = np.concatenate([X_text_1_vm, X_text_2_vm], axis=1)

In [85]:
X_text_vm.shape

(7227, 200)

In [80]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [86]:
y = data['label'].values
print(y.shape)

(7227,)


In [87]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_vm, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

              precision    recall  f1-score   support

          -1       0.47      0.50      0.48       629
           0       0.46      0.62      0.53       737
           1       0.35      0.10      0.15       441

    accuracy                           0.45      1807
   macro avg       0.42      0.41      0.39      1807
weighted avg       0.43      0.45      0.42      1807



In [88]:
cross_val_score(clf, X_text_vm, y, scoring='f1_micro', cv=10).mean()

0.42046974858641645

## Задание 2. Реализовать алгоритм Леска и проверить его на реальном датасете

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

In [90]:
# запустите если не установлен ворднет
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

Реализуйте его

In [139]:
stopw = set(stopwords.words('english'))
def tok_en(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stopw]
    words = [word for word in words if word]

    return words

def norm_en(text):
    
    words = tok_en(text)
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

In [150]:
def lesk(word, sentence):
    bestsense = 0
    maxoverlap = 0
    overlaps = []
    
    w_synsets = wn.synsets(word)
    
    # 所有的定义[[w, w, ...], [w, w, ...], ...]
    defs = [synset.definition().split() for synset in wn.synsets(word)]
    sent = [w.strip(punct) for w in sentence if w.strip(punct) and w not in stopw]
    
    for i, syns in enumerate(w_synsets):
        defi = norm_en(syns.definition())
        # defi = syns.definition().split()
        # defi = [w.strip(punct) for w in defi if w.strip(punct) and w not in stopw]
        overlap = len([i for i in sent if i in defi])
        overlaps.append(overlap)
    
        maxoverlap = max(overlaps)
        bestsense = overlaps.index(maxoverlap)
        
    return bestsense

Работать функция должна как-то так:

In [147]:
lesk('day', 'time for a planet to make a complete rotation on its axis'.split())

6

In [138]:
wn.synsets('day')[6].definition()

'the period of time taken by a particular planet (e.g. Mars) to make a complete rotation on its axis'

**Проверьте насколько хорошо работает такой метод на реальном датасете.** http://lcl.uniroma1.it/wsdeval/ - большой фреймворк для оценки WSD. Там много данных и я взял кусочек, чтобы не было проблем с памятью

In [98]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

In [102]:
corpus_wsd[:2]

[[['', 'how', 'How'],
  ['long%3:00:02::', 'long', 'long'],
  ['', 'have', 'has'],
  ['', 'it', 'it'],
  ['be%2:42:03::', 'be', 'been'],
  ['', 'since', 'since'],
  ['', 'you', 'you'],
  ['review%2:31:00::', 'review', 'reviewed'],
  ['', 'the', 'the'],
  ['objective%1:09:00::', 'objective', 'objectives'],
  ['', 'of', 'of'],
  ['', 'you', 'your'],
  ['benefit%1:21:00::', 'benefit', 'benefit'],
  ['', 'and', 'and'],
  ['service%1:04:07::', 'service', 'service'],
  ['program%1:09:01::', 'program', 'program'],
  ['', '?', '?']],
 [['', 'have', 'Have'],
  ['', 'you', 'you'],
  ['permit%2:41:00::', 'permit', 'permitted'],
  ['', 'it', 'it'],
  ['', 'to', 'to'],
  ['become%2:42:01::', 'become', 'become'],
  ['', 'a', 'a'],
  ['giveaway%1:21:00::', 'giveaway', 'giveaway'],
  ['program%1:09:01::', 'program', 'program'],
  ['rather%4:02:02::', 'rather', 'rather'],
  ['', 'than', 'than'],
  ['', 'one', 'one'],
  ['', 'that', 'that'],
  ['have%2:42:00::', 'have', 'has'],
  ['', 'the', 'the'],
  [

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

In [154]:
counter = 0
counter_true = 0
for sent in corpus_wsd[:5000]:
    sentence = [w[1] for w in sent]
    for word in sent:
        if word[0]:
            counter += 1
            true_syn = wn.lemma_from_key(word[0]).synset()
            true = wn.synsets(word[1]).index(true_syn)
            pred = lesk(word[1], sentence)
            if true == pred:
                counter_true += 1

In [155]:
acc = counter_true / counter
acc

0.4332027947131859

### Дополнительный балл

Если хотите заработать дополнительный балл, попробуйте улучшить алгоритм Леска любым способом (например, использовать расстояние редактирования вместо пересечения или даже вставить машинное обучение)